# SWELL-KW LSTM

Adapted from Microsoft's notebooks, available at https://github.com/microsoft/EdgeML authored by Dennis et al.

## Imports

In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import os
import datetime as datetime
import pickle as pkl
import pathlib

## DataFrames from CSVs

In [224]:
df = pd.read_csv('/home/sf/data/SWELL-KW/EDA_New.csv', index_col=0)

## Preprocessing 

In [226]:
df['condition'].replace('time pressure', 'interruption', inplace=True)

In [227]:
df['condition'].replace('interruption', 1, inplace=True)
df['condition'].replace('no stress', 0, inplace=True)

## Split Ground Truth 

In [188]:
df[df['condition']==1]['RANGE'].mean()

0.027338218248412812

In [229]:
filtered_train = df.drop(['condition'], axis=1)
filtered_target = df['condition']

In [231]:
print(filtered_target.shape)
print(filtered_train.shape)

(104040,)
(104040, 22)


In [232]:
pd.value_counts(filtered_target)

0    52695
1    51345
Name: condition, dtype: int64

In [233]:
y = filtered_target.values.reshape(-1, 20)

## Convert to 3D - (Bags, Timesteps, Features)

In [234]:
len(filtered_train.columns)

22

In [235]:
x = filtered_train.values
print(x.shape)
x = x.reshape(int(len(x) / 20), 20, 22)
print(x.shape)

(104040, 22)
(5202, 20, 22)


## Filter Overlapping Bags

In [236]:
# filtering bags that overlap with another class
bags_to_remove = []
for i in range(len(y)):
    if len(set(y[i])) > 1:
        bags_to_remove.append(i)
print(bags_to_remove)

[120, 198, 289, 408, 410, 415, 544, 667, 776, 894, 908, 911, 913, 1019, 1103, 1117, 1147, 1150, 1191, 1305, 1440, 1549, 1643, 1660, 1685, 1805, 1937, 2042, 2175, 2179, 2183, 2184, 2384, 2490, 2566, 2571, 2594, 2597, 2637, 2638, 2761, 2810, 2918, 2990, 3004, 3008, 3015, 3056, 3164, 3236, 3244, 3274, 3277, 3318, 3433, 3546, 3556, 3560, 3562, 3563, 3564, 3676, 3786, 3796, 3800, 3839, 3840, 3841, 3956, 4051, 4081, 4084, 4125, 4126, 4232, 4310, 4411, 4527, 4544, 4546, 4549, 4554, 4559, 4564, 4687, 4789, 4895, 4968, 4985, 4989, 5008, 5131]


In [237]:
x = np.delete(x, bags_to_remove, axis=0)
y = np.delete(y, bags_to_remove, axis=0)

In [238]:
x.shape 

(5110, 20, 22)

In [239]:
y.shape

(5110, 20)

## Categorical Representation 

In [240]:
one_hot_list = []
for i in range(len(y)):
    one_hot_list.append(set(y[i]).pop())

In [241]:
categorical_y_ver = one_hot_list
categorical_y_ver = np.array(categorical_y_ver)

In [242]:
categorical_y_ver.shape

(5110,)

In [243]:
x.shape[1]

20

In [244]:
def one_hot(y, numOutput):
    y = np.reshape(y, [-1])
    ret = np.zeros([y.shape[0], numOutput])
    for i, label in enumerate(y):
        ret[i, label] = 1
    return ret

## Extract 3D Normalized Data with Validation Set

In [245]:
from sklearn.model_selection import train_test_split
import pathlib

In [246]:
x_train_val_combined, x_test, y_train_val_combined, y_test = train_test_split(x, categorical_y_ver, test_size=0.20, random_state=42)

In [247]:
y_test

array([1, 1, 0, ..., 1, 0, 1])

In [248]:
extractedDir = '/home/sf/data/SWELL-KW/'
# def generateData(extractedDir):
# x_train_val_combined, x_test, y_train_val_combined, y_test = readData(extractedDir)
timesteps = x_train_val_combined.shape[-2] #128/256
feats = x_train_val_combined.shape[-1]  #16

trainSize = int(x_train_val_combined.shape[0]*0.9) #6566
x_train, x_val = x_train_val_combined[:trainSize], x_train_val_combined[trainSize:] 
y_train, y_val = y_train_val_combined[:trainSize], y_train_val_combined[trainSize:]

# normalization
x_train = np.reshape(x_train, [-1, feats])
mean = np.mean(x_train, axis=0)
std = np.std(x_train, axis=0)

# normalize train
x_train = x_train - mean
x_train = x_train / std
x_train = np.reshape(x_train, [-1, timesteps, feats])

# normalize val
x_val = np.reshape(x_val, [-1, feats])
x_val = x_val - mean
x_val = x_val / std
x_val = np.reshape(x_val, [-1, timesteps, feats])

# normalize test
x_test = np.reshape(x_test, [-1, feats])
x_test = x_test - mean
x_test = x_test / std
x_test = np.reshape(x_test, [-1, timesteps, feats])

# shuffle test, as this was remaining
idx = np.arange(len(x_test))
np.random.shuffle(idx)
x_test = x_test[idx]
y_test = y_test[idx]

In [249]:
# one-hot encoding of labels
numOutput = 2
y_train = one_hot(y_train, numOutput)
y_val = one_hot(y_val, numOutput)
y_test = one_hot(y_test, numOutput)
extractedDir += '/'

pathlib.Path(extractedDir + 'RAW').mkdir(parents=True, exist_ok = True)

np.save(extractedDir + "RAW/x_train", x_train)
np.save(extractedDir + "RAW/y_train", y_train)
np.save(extractedDir + "RAW/x_test", x_test)
np.save(extractedDir + "RAW/y_test", y_test)
np.save(extractedDir + "RAW/x_val", x_val)
np.save(extractedDir + "RAW/y_val", y_val)

print(extractedDir)

/home/sf/data/SWELL-KW//


In [250]:
ls /home/sf/data/SWELL-KW/RAW

x_test.npy  x_train.npy  x_val.npy  y_test.npy  y_train.npy  y_val.npy


In [251]:
np.load('//home/sf/data/SWELL-KW/RAW/x_train.npy').shape

(3679, 20, 22)

## Make 4D EMI Data (Bags, Subinstances, Subinstance Length, Features)

In [2]:
def loadData(dirname):
    x_train = np.load(dirname + '/' + 'x_train.npy')
    y_train = np.load(dirname + '/' + 'y_train.npy')
    x_test = np.load(dirname + '/' + 'x_test.npy')
    y_test = np.load(dirname + '/' + 'y_test.npy')
    x_val = np.load(dirname + '/' + 'x_val.npy')
    y_val = np.load(dirname + '/' + 'y_val.npy')
    return x_train, y_train, x_test, y_test, x_val, y_val

In [3]:
def bagData(X, Y, subinstanceLen, subinstanceStride):
    numClass = 2
    numSteps = 20
    numFeats = 22
    assert X.ndim == 3
    assert X.shape[1] == numSteps
    assert X.shape[2] == numFeats
    assert subinstanceLen <= numSteps
    assert subinstanceLen > 0
    assert subinstanceStride <= numSteps
    assert subinstanceStride >= 0
    assert len(X) == len(Y)
    assert Y.ndim == 2
    assert Y.shape[1] == numClass
    x_bagged = []
    y_bagged = []
    for i, point in enumerate(X[:, :, :]):
        instanceList = []
        start = 0
        end = subinstanceLen
        while True:
            x = point[start:end, :]
            if len(x) < subinstanceLen:
                x_ = np.zeros([subinstanceLen, x.shape[1]])
                x_[:len(x), :] = x[:, :]
                x = x_
            instanceList.append(x)
            if end >= numSteps:
                break
            start += subinstanceStride
            end += subinstanceStride
        bag = np.array(instanceList)
        numSubinstance = bag.shape[0]
        label = Y[i]
        label = np.argmax(label)
        labelBag = np.zeros([numSubinstance, numClass])
        labelBag[:, label] = 1
        x_bagged.append(bag)
        label = np.array(labelBag)
        y_bagged.append(label)
    return np.array(x_bagged), np.array(y_bagged)

In [4]:
def makeEMIData(subinstanceLen, subinstanceStride, sourceDir, outDir):
    x_train, y_train, x_test, y_test, x_val, y_val = loadData(sourceDir)
    x, y = bagData(x_train, y_train, subinstanceLen, subinstanceStride)
    np.save(outDir + '/x_train.npy', x)
    np.save(outDir + '/y_train.npy', y)
    print('Num train %d' % len(x))
    x, y = bagData(x_test, y_test, subinstanceLen, subinstanceStride)
    np.save(outDir + '/x_test.npy', x)
    np.save(outDir + '/y_test.npy', y)
    print('Num test %d' % len(x))
    x, y = bagData(x_val, y_val, subinstanceLen, subinstanceStride)
    np.save(outDir + '/x_val.npy', x)
    np.save(outDir + '/y_val.npy', y)
    print('Num val %d' % len(x))

In [5]:
subinstanceLen = 8
subinstanceStride = 3
extractedDir = '/home/sf/data/SWELL-KW'
from os import mkdir
#mkdir('/home/sf/data/SWELL-KW/8_3')
rawDir = extractedDir + '/RAW'
sourceDir = rawDir
outDir = extractedDir + '/%d_%d/' % (subinstanceLen, subinstanceStride)
makeEMIData(subinstanceLen, subinstanceStride, sourceDir, outDir)

Num train 3679
Num test 1022
Num val 409


In [6]:
np.load('/home/sf/data/SWELL-KW/8_3/y_train.npy').shape

(3679, 5, 2)

In [8]:
from edgeml.graph.rnn import EMI_DataPipeline
from edgeml.graph.rnn import EMI_BasicLSTM, EMI_FastGRNN, EMI_FastRNN, EMI_GRU
from edgeml.trainer.emirnnTrainer import EMI_Trainer, EMI_Driver
import edgeml.utils

In [9]:
def lstm_experiment_generator(params, path = './DSAAR/64_16/'):
    """
        Function that will generate the experiments to be run.
        Inputs : 
        (1) Dictionary params, to set the network parameters.
        (2) Name of the Model to be run from [EMI-LSTM, EMI-FastGRNN, EMI-GRU]
        (3) Path to the dataset, where the csv files are present.
    """
    
    #Copy the contents of the params dictionary.
    lstm_dict = {**params}
    
    #---------------------------PARAM SETTING----------------------#
    
    # Network parameters for our LSTM + FC Layer
    NUM_HIDDEN = params["NUM_HIDDEN"]
    NUM_TIMESTEPS = params["NUM_TIMESTEPS"]
    ORIGINAL_NUM_TIMESTEPS = params["ORIGINAL_NUM_TIMESTEPS"]
    NUM_FEATS = params["NUM_FEATS"]
    FORGET_BIAS = params["FORGET_BIAS"]
    NUM_OUTPUT = params["NUM_OUTPUT"]
    USE_DROPOUT = True if (params["USE_DROPOUT"] == 1) else False
    KEEP_PROB = params["KEEP_PROB"]

    # For dataset API
    PREFETCH_NUM = params["PREFETCH_NUM"]
    BATCH_SIZE = params["BATCH_SIZE"]

    # Number of epochs in *one iteration*
    NUM_EPOCHS = params["NUM_EPOCHS"]
    # Number of iterations in *one round*. After each iteration,
    # the model is dumped to disk. At the end of the current
    # round, the best model among all the dumped models in the
    # current round is picked up..
    NUM_ITER = params["NUM_ITER"]
    # A round consists of multiple training iterations and a belief
    # update step using the best model from all of these iterations
    NUM_ROUNDS = params["NUM_ROUNDS"]
    LEARNING_RATE = params["LEARNING_RATE"]

    # A staging direcory to store models
    MODEL_PREFIX = params["MODEL_PREFIX"]
    
    #----------------------END OF PARAM SETTING----------------------#
    
    #----------------------DATA LOADING------------------------------#
    
    x_train, y_train = np.load(path + 'x_train.npy'), np.load(path + 'y_train.npy')
    x_test, y_test = np.load(path + 'x_test.npy'), np.load(path + 'y_test.npy')
    x_val, y_val = np.load(path + 'x_val.npy'), np.load(path + 'y_val.npy')

    # BAG_TEST, BAG_TRAIN, BAG_VAL represent bag_level labels. These are used for the label update
    # step of EMI/MI RNN
    BAG_TEST = np.argmax(y_test[:, 0, :], axis=1)
    BAG_TRAIN = np.argmax(y_train[:, 0, :], axis=1)
    BAG_VAL = np.argmax(y_val[:, 0, :], axis=1)
    NUM_SUBINSTANCE = x_train.shape[1]
    print("x_train shape is:", x_train.shape)
    print("y_train shape is:", y_train.shape)
    print("x_test shape is:", x_val.shape)
    print("y_test shape is:", y_val.shape)
    
    #----------------------END OF DATA LOADING------------------------------#    
    
    #----------------------COMPUTATION GRAPH--------------------------------#
    
    # Define the linear secondary classifier
    def createExtendedGraph(self, baseOutput, *args, **kwargs):
        W1 = tf.Variable(np.random.normal(size=[NUM_HIDDEN, NUM_OUTPUT]).astype('float32'), name='W1')
        B1 = tf.Variable(np.random.normal(size=[NUM_OUTPUT]).astype('float32'), name='B1')
        y_cap = tf.add(tf.tensordot(baseOutput, W1, axes=1), B1, name='y_cap_tata')
        self.output = y_cap
        self.graphCreated = True

    def restoreExtendedGraph(self, graph, *args, **kwargs):
        y_cap = graph.get_tensor_by_name('y_cap_tata:0')
        self.output = y_cap
        self.graphCreated = True

    def feedDictFunc(self, keep_prob=None, inference=False, **kwargs):
        if inference is False:
            feedDict = {self._emiGraph.keep_prob: keep_prob}
        else:
            feedDict = {self._emiGraph.keep_prob: 1.0}
        return feedDict

    EMI_BasicLSTM._createExtendedGraph = createExtendedGraph
    EMI_BasicLSTM._restoreExtendedGraph = restoreExtendedGraph

    if USE_DROPOUT is True:
        EMI_Driver.feedDictFunc = feedDictFunc
    
    inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT)
    emiLSTM = EMI_BasicLSTM(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS,
                            forgetBias=FORGET_BIAS, useDropout=USE_DROPOUT)
    emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy',
                             stepSize=LEARNING_RATE)
    
    tf.reset_default_graph()
    g1 = tf.Graph()    
    with g1.as_default():
        # Obtain the iterators to each batch of the data
        x_batch, y_batch = inputPipeline()
        # Create the forward computation graph based on the iterators
        y_cap = emiLSTM(x_batch)
        # Create loss graphs and training routines
        emiTrainer(y_cap, y_batch)
        
    #------------------------------END OF COMPUTATION GRAPH------------------------------#
    
    #-------------------------------------EMI DRIVER-------------------------------------#
        
    with g1.as_default():
        emiDriver = EMI_Driver(inputPipeline, emiLSTM, emiTrainer)

    emiDriver.initializeSession(g1)
    y_updated, modelStats = emiDriver.run(numClasses=NUM_OUTPUT, x_train=x_train,
                                          y_train=y_train, bag_train=BAG_TRAIN,
                                          x_val=x_val, y_val=y_val, bag_val=BAG_VAL,
                                          numIter=NUM_ITER, keep_prob=KEEP_PROB,
                                          numRounds=NUM_ROUNDS, batchSize=BATCH_SIZE,
                                          numEpochs=NUM_EPOCHS, modelPrefix=MODEL_PREFIX,
                                          fracEMI=0.5, updatePolicy='top-k', k=1)
    
    #-------------------------------END OF EMI DRIVER-------------------------------------#
    
    #-----------------------------------EARLY SAVINGS-------------------------------------#
    
    """
        Early Prediction Policy: We make an early prediction based on the predicted classes
        probability. If the predicted class probability > minProb at some step, we make
        a prediction at that step.
    """
    def earlyPolicy_minProb(instanceOut, minProb, **kwargs):
        assert instanceOut.ndim == 2
        classes = np.argmax(instanceOut, axis=1)
        prob = np.max(instanceOut, axis=1)
        index = np.where(prob >= minProb)[0]
        if len(index) == 0:
            assert (len(instanceOut) - 1) == (len(classes) - 1)
            return classes[-1], len(instanceOut) - 1
        index = index[0]
        return classes[index], index

    def getEarlySaving(predictionStep, numTimeSteps, returnTotal=False):
        predictionStep = predictionStep + 1
        predictionStep = np.reshape(predictionStep, -1)
        totalSteps = np.sum(predictionStep)
        maxSteps = len(predictionStep) * numTimeSteps
        savings = 1.0 - (totalSteps / maxSteps)
        if returnTotal:
            return savings, totalSteps
        return savings
    
    #--------------------------------END OF EARLY SAVINGS---------------------------------#
    
    #----------------------------------------BEST MODEL-----------------------------------#
    
    k = 2
    predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                                   minProb=0.99, keep_prob=1.0)
    bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
    print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))
    mi_savings = (1 - NUM_TIMESTEPS / ORIGINAL_NUM_TIMESTEPS)
    emi_savings = getEarlySaving(predictionStep, NUM_TIMESTEPS)
    total_savings = mi_savings + (1 - mi_savings) * emi_savings
    print('Savings due to MI-RNN : %f' % mi_savings)
    print('Savings due to Early prediction: %f' % emi_savings)
    print('Total Savings: %f' % (total_savings))
    
    #Store in the dictionary.
    lstm_dict["k"] = k
    lstm_dict["accuracy"] = np.mean((bagPredictions == BAG_TEST).astype(int))
    lstm_dict["total_savings"] = total_savings
    lstm_dict["y_test"] = BAG_TEST
    lstm_dict["y_pred"] = bagPredictions
    
    # A slightly more detailed analysis method is provided. 
    df = emiDriver.analyseModel(predictions, BAG_TEST, NUM_SUBINSTANCE, NUM_OUTPUT)
    print (tabulate(df, headers=list(df.columns), tablefmt='grid'))
    
    lstm_dict["detailed analysis"] = df
    #----------------------------------END OF BEST MODEL-----------------------------------#
    
    #----------------------------------PICKING THE BEST MODEL------------------------------#
    
    devnull = open(os.devnull, 'r')
    for val in modelStats:
        round_, acc, modelPrefix, globalStep = val
        emiDriver.loadSavedGraphToNewSession(modelPrefix, globalStep, redirFile=devnull)
        predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                                   minProb=0.99, keep_prob=1.0)

        bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
        print("Round: %2d, Validation accuracy: %.4f" % (round_, acc), end='')
        print(', Test Accuracy (k = %d): %f, ' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))), end='')
        print('Additional savings: %f' % getEarlySaving(predictionStep, NUM_TIMESTEPS)) 
        
    
    #-------------------------------END OF PICKING THE BEST MODEL--------------------------#

    return lstm_dict

In [10]:
def experiment_generator(params, path, model = 'lstm'):
    
    
    if (model == 'lstm'): return lstm_experiment_generator(params, path)
    elif (model == 'fastgrnn'): return fastgrnn_experiment_generator(params, path)
    elif (model == 'gru'): return gru_experiment_generator(params, path)
    elif (model == 'baseline'): return baseline_experiment_generator(params, path)
    
    return 

In [11]:
pwd

'/home/sf/data/EdgeML/tf/examples/EMI-RNN'

In [12]:
import tensorflow as tf

In [13]:
## Baseline EMI-LSTM

dataset = 'SWELL-KW'
path = '/home/sf/data/SWELL-KW/8_3/'

#Choose model from among [lstm, fastgrnn, gru]
model = 'lstm'

# Dictionary to set the parameters.
params = {
    "NUM_HIDDEN" : 128,
    "NUM_TIMESTEPS" : 8, #subinstance length.
    "ORIGINAL_NUM_TIMESTEPS" : 20,
    "NUM_FEATS" : 22,
    "FORGET_BIAS" : 1.0,
    "NUM_OUTPUT" : 2,
    "USE_DROPOUT" : 1, # '1' -> True. '0' -> False
    "KEEP_PROB" : 0.75,
    "PREFETCH_NUM" : 5,
    "BATCH_SIZE" : 32,
    "NUM_EPOCHS" : 2,
    "NUM_ITER" : 4,
    "NUM_ROUNDS" : 30,
    "LEARNING_RATE" : 0.001,
    "FRAC_EMI" : 0.5,
    "MODEL_PREFIX" : '/home/sf/data/SWELL-KW/models/model-lstm'
}

#Preprocess data, and load the train,test and validation splits.
lstm_dict = lstm_experiment_generator(params, path)

#Create the directory to store the results of this run.

dirname = "" + model
pathlib.Path(dirname).mkdir(parents=True, exist_ok=True)
print ("Results for this run have been saved at" , dirname, ".")

now = datetime.datetime.now()
filename = list((str(now.year),"-",str(now.month),"-",str(now.day),"|",str(now.hour),"-",str(now.minute)))
filename = ''.join(filename)

#Save the dictionary containing the params and the results.
pkl.dump(lstm_dict,open(dirname + "/lstm_dict_" + filename + ".pkl",mode='wb'))

x_train shape is: (3679, 5, 8, 22)
y_train shape is: (3679, 5, 2)
x_test shape is: (409, 5, 8, 22)
y_test shape is: (409, 5, 2)
Update policy: top-k
Training with MI-RNN loss for 15 rounds
Round: 0
Epoch   1 Batch   110 (  225) Loss 0.08400 Acc 0.59375 | Val acc 0.58924 | Model saved to /home/sf/data/SWELL-KW/models/model-lstm, global_step 1000
Epoch   1 Batch   110 (  225) Loss 0.08510 Acc 0.57500 | Val acc 0.63081 | Model saved to /home/sf/data/SWELL-KW/models/model-lstm, global_step 1001
Epoch   1 Batch   110 (  225) Loss 0.07742 Acc 0.61250 | Val acc 0.66015 | Model saved to /home/sf/data/SWELL-KW/models/model-lstm, global_step 1002
Epoch   1 Batch   110 (  225) Loss 0.07279 Acc 0.67500 | Val acc 0.66504 | Model saved to /home/sf/data/SWELL-KW/models/model-lstm, global_step 1003
INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/models/model-lstm-1003
Round: 1
Epoch   1 Batch   110 (  225) Loss 0.06990 Acc 0.71875 | Val acc 0.67971 | Model saved to /home/sf/data/SWELL

Epoch   1 Batch   110 (  225) Loss 0.03921 Acc 0.85000 | Val acc 0.81418 | Model saved to /home/sf/data/SWELL-KW/models/model-lstm, global_step 1047
INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/models/model-lstm-1044
Round: 12
Epoch   1 Batch   110 (  225) Loss 0.04475 Acc 0.81875 | Val acc 0.82641 | Model saved to /home/sf/data/SWELL-KW/models/model-lstm, global_step 1048
Epoch   1 Batch   110 (  225) Loss 0.03966 Acc 0.85625 | Val acc 0.83130 | Model saved to /home/sf/data/SWELL-KW/models/model-lstm, global_step 1049
Epoch   1 Batch   110 (  225) Loss 0.03139 Acc 0.90000 | Val acc 0.83619 | Model saved to /home/sf/data/SWELL-KW/models/model-lstm, global_step 1050
Epoch   1 Batch   110 (  225) Loss 0.04275 Acc 0.81250 | Val acc 0.81663 | Model saved to /home/sf/data/SWELL-KW/models/model-lstm, global_step 1051
INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/models/model-lstm-1050
Round: 13
Epoch   1 Batch   110 (  225) Loss 0.03359 Acc 0.87500 | Va

Epoch   1 Batch   110 (  225) Loss 0.37442 Acc 0.91875 | Val acc 0.84841 | Model saved to /home/sf/data/SWELL-KW/models/model-lstm, global_step 1094
Epoch   1 Batch   110 (  225) Loss 0.32392 Acc 0.91875 | Val acc 0.84108 | Model saved to /home/sf/data/SWELL-KW/models/model-lstm, global_step 1095
INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/models/model-lstm-1094
Round: 24
Epoch   1 Batch   110 (  225) Loss 0.32550 Acc 0.90625 | Val acc 0.83863 | Model saved to /home/sf/data/SWELL-KW/models/model-lstm, global_step 1096
Epoch   1 Batch   110 (  225) Loss 0.34901 Acc 0.90625 | Val acc 0.83619 | Model saved to /home/sf/data/SWELL-KW/models/model-lstm, global_step 1097
Epoch   1 Batch   110 (  225) Loss 0.34062 Acc 0.90625 | Val acc 0.84108 | Model saved to /home/sf/data/SWELL-KW/models/model-lstm, global_step 1098
Epoch   1 Batch   110 (  225) Loss 0.33095 Acc 0.90625 | Val acc 0.84597 | Model saved to /home/sf/data/SWELL-KW/models/model-lstm, global_step 1099
INFO:ten

Round:  5, Validation accuracy: 0.8044, Test Accuracy (k = 2): 0.793542, Additional savings: 0.037769
INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/models/model-lstm-1027
Round:  6, Validation accuracy: 0.8337, Test Accuracy (k = 2): 0.817025, Additional savings: 0.049315
INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/models/model-lstm-1031
Round:  7, Validation accuracy: 0.8289, Test Accuracy (k = 2): 0.826810, Additional savings: 0.054795
INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/models/model-lstm-1032
Round:  8, Validation accuracy: 0.8362, Test Accuracy (k = 2): 0.825832, Additional savings: 0.060372
INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/models/model-lstm-1039
Round:  9, Validation accuracy: 0.8264, Test Accuracy (k = 2): 0.821918, Additional savings: 0.068249
INFO:tensorflow:Restoring parameters from /home/sf/data/SWELL-KW/models/model-lstm-1042
Round: 10, Validation accuracy: 0.8289, Test Accur